In [14]:
#https://www.dicomlibrary.com/dicom/dicom-tags/
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import json
from sqlalchemy import create_engine, inspect, Table, Column
import pandas as pd
from ilxutils.args_reader import read_args
from ilxutils.scicrunch_client import scicrunch
import sys
import time
from ilxutils.interlex_sql import interlex_sql
import re
from collections import defaultdict
from IPython.core.display import display
import subprocess as sb
import pickle
from pathlib import Path as p
import requests as r
from goatools import obo_parser

def degrade(var):
    def helper(s):
        return re.sub("\(|\)|'|,", "", s).lower().strip()
    if type(var)!=list:
        if var:
            return helper(var)
        else:
            return var
    else:
        return [helper(v) if v else v for v in var]

def get(url):
    req = r.get(url)
    if req.raise_for_status():
        sys.exit('failed at '+ url)
    try:
        return req.json()
    except:
        return req.text
    
def cj(data, output):
    try:
        json.dump(data, open(output+'.json', 'w'), indent=4)
        print('Complete')
    except:
        sys.exit('Data not json formatable')
        
def oj(infile):
    try:
        return json.load(open(infile, 'r'))
    except:
        sys.exit('File does not exist')
        
args = read_args(api_key= p.home() / 'keys/production_api_scicrunch_key.txt', db_url= p.home() / 'keys/production_engine_scicrunch_key.txt', production=True)
sql = interlex_sql(db_url=args.db_url)
sci = scicrunch(api_key=args.api_key, base_path=args.base_path, db_url=args.db_url)

In [15]:
ex = sql.get_existing_ids()

In [16]:
ex[ex.curie.str.contains('OEN')].shape

(249, 9)

In [17]:
data = []
for i, row in ex[ex.curie.str.contains('OEN')].iterrows():
    if row.curie.split(':')[0] == 'OEN':
        data.append({'id':row.tid, 'existing_ids':{'curie':row.curie,'iri':row.iri, 'delete':True}})
print(data)
#cj(data, 'deleting_oen_existing_ids')

[]
